# Feature Extraction

This sections will cover how to extract regionprops features from an image using `ngio`, `skimage`. Moreover we will also write the features to a table in the ome-zarr container.

# Step 1: Open the OME-Zarr Container

In [ ]:
from pathlib import Path

from ngio import open_ome_zarr_container
from ngio.utils import download_ome_zarr_dataset

# Download the dataset
download_dir = Path(".").absolute().parent.parent / "data"
hcs_path = download_ome_zarr_dataset("CardiomyocyteTinyMip", download_dir=download_dir)
image_path = hcs_path / "B" / "03" / "0"

# Open the ome-zarr container
ome_zarr = open_ome_zarr_container(image_path)

## Step 2: Compute the Regionprops Features

In [ ]:
import pandas as pd
from skimage import measure

from ngio.tables import FeatureTable

# First we will need the image object and the FOVs table
nuclei = ome_zarr.get_label("nuclei")
image_data = ome_zarr.get_image(pixel_size=nuclei.pixel_size).get_as_numpy(
    axes_order=["z", "y", "x", "c"]
)
nuclei_data = nuclei.get_as_numpy(axes_order=["z", "y", "x"])

feat_table = measure.regionprops_table(
    label_image=nuclei_data,
    intensity_image=image_data,
    properties=["label", "area", "mean_intensity", "max_intensity", "min_intensity"],
)

feat_table = pd.DataFrame(feat_table)
feat_table = feat_table.set_index("label")

# Convert to a FeatureTable
feature_table = FeatureTable(table_data=feat_table, reference_label="nuclei")

# Save the Table in the ome-zarr container
ome_zarr.add_table("nuclei_regionprops", feature_table)

## Sanity Check: Read the Table back

In [ ]:
ome_zarr.get_table("nuclei_regionprops").lazy_frame.collect()